In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from libpysal.weights import Kernel
from esda.moran import Moran

from scipy import stats

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_predict, cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import minmax_scale

In [ ]:
# set seaborn theme
sns.set_theme(style='darkgrid')

In [ ]:
loc_gdf = gpd.read_file('data/AQMS_loc.shp')

# Initialise

In [ ]:
# Read in all the data

AQMS_df = pd.read_csv('data/hourly.csv')
Rd_gdf = gpd.read_file('data/london_Road.shp')
Gsp_gdf = gpd.read_file('data/LD_GreenSpace.shp')
cond = pd.read_csv('data/cond_hourly.csv')

In [ ]:
# reindex loc_gdf and set buffer zones around each site (1km)

loc_gdf = loc_gdf.set_index('siteid')
loc_gdf['buffer_1km'] = loc_gdf['geometry'].buffer(1000)

In [ ]:
Rd_gdf.head()

In [ ]:
for typ in Rd_gdf['function'].unique():
    print('Number of ' + typ + ': ', Rd_gdf[Rd_gdf['function'] == typ].shape[0])

In [ ]:
# Get all green spaces
Gsp = Gsp_gdf['geometry'].unary_union

# Get all types of roads
Rd = {}
for typ in Rd_gdf['function'].unique():
    Rd[typ] = Rd_gdf[Rd_gdf['function'] == typ].loc[:, 'geometry'].unary_union
Rd

In [ ]:
del Gsp_gdf, Rd_gdf

In [ ]:
loc_gdf['Gsp'] = loc_gdf['buffer_1km'].intersection(Gsp)
for key in Rd.keys():
    loc_gdf[key] = loc_gdf['buffer_1km'].intersection(Rd[key])

loc_gdf.head()

In [ ]:
del Rd

In [ ]:
# Rename columns
loc_gdf.rename(columns={'Restricted Local Access Road': 'RLA_Rd', 
                        'Minor Road': 'Mi_Rd',
                        'A Road': 'A_Rd',
                        'Local Road': 'L_Rd',
                        'B Road': 'B_Rd',
                        'Local Access Road': 'LA_Rd',
                        'Secondary Access Road': 'SA_Rd',
                        'Motorway': 'Mo_Rd'}, inplace=True)

loc_gdf.columns

In [ ]:
# Get all near-road green spaces
Rd_type = loc_gdf.columns[4:]
for col in Rd_type:
    loc_gdf['n'+col+'_Gsp'] = loc_gdf['Gsp'].intersection(loc_gdf[col].buffer(50))

In [ ]:
loc_gdf.head()

In [ ]:
# london boundary read in
london = gpd.read_file('data/london_boundary.shp')

In [ ]:
# visualise all the sites on the map
fig,ax = plt.subplots(1, figsize=(15,13))

london.plot(color='lightgrey', ax=ax)
loc_gdf['buffer_1km'].plot(color='silver', ax=ax)
loc_gdf['geometry'].plot(markersize=10, marker='^', color='blue', 
                         label='Air quality monitoring site', ax=ax)

ax.axis('off')

legend=ax.legend(loc='best',shadow=True,fontsize=15)

#plt.savefig('sample1.png',facecolor='black',dpi=500)
plt.show()

There are some buffers that seem to be very close to each other.

In [ ]:
# add a column that specifies the shortest distance of a site to its nearest neighbour
loc_gdf['min_dis'] = pd.Series(dtype='float64')
for index, row in loc_gdf.iterrows():
    dis = []
    for i, v in loc_gdf['geometry'].iteritems():
        dis.append(row['geometry'].distance(v))
    dis.remove(0)
    loc_gdf.loc[index, 'min_dis'] = min(dis)

In [ ]:
# list sites that are close to each other (within 1.5km)
loc_gdf[loc_gdf['min_dis']<=1500]

In [ ]:
# check their readings' descriptive statistics
AQMS_df[AQMS_df['Site'].isin(['BL0', 'CD9', 'GR4', 'GB0'])].groupby('Site').describe()

In [ ]:
stats.ttest_rel(AQMS_df[AQMS_df['Site']=='BL0'].Value.values,
                AQMS_df[AQMS_df['Site']=='CD9'].Value.values)

In [ ]:
stats.ttest_rel(AQMS_df[AQMS_df['Site']=='GR4'].Value.values,
                AQMS_df[AQMS_df['Site']=='GB0'].Value.values)

Both indicate that we should reject H0, meaning the two datasets are statistically significantly different.

In [ ]:
# revmove them from the list
#loc_gdf.drop(['BL0','GR4'], inplace=True)

In [ ]:
# get areas and edge lengths of green spaces 
loc_gdf['Gsp_area'] = loc_gdf['Gsp'].area
loc_gdf['Gsp_edge'] = loc_gdf['Gsp'].length

In [ ]:
# get road lengths of each type and nRd gsp area percentages
for col in Rd_type:
    loc_gdf[col+'_len'] = loc_gdf[col].length
    loc_gdf['pct_n'+col+'_Gsp'] = loc_gdf['n'+ col +'_Gsp'].area / loc_gdf['Gsp_area'] * 100

In [ ]:
loc_gdf.columns

In [ ]:
loc_gdf[[col+'_len' for col in Rd_type]].sum(axis=1)

In [ ]:
loc_gdf['Gsp_per_tRd_len'] = loc_gdf['Gsp_area'] / loc_gdf[[col+'_len' for col in Rd_type]].sum(axis=1)

In [ ]:
loc_gdf.info()

In [ ]:
exp_names = loc_gdf.columns[21:].tolist()
exp_names

In [ ]:
# merge PM reading and site geogemetry data
df = pd.merge(AQMS_df, loc_gdf, left_on='Site', right_index=True)
df.info()

In [ ]:
# drop irrelevant columns
df.drop(['sitename', 'geometry', 'buffer_1km', 'Gsp', 'min_dis'], axis=1, inplace=True)
df.drop(Rd_type , axis=1, inplace=True)
df.drop(['n'+rd+'_Gsp' for rd in Rd_type], axis=1, inplace=True)

df.info()

In [ ]:
# merge with conditional variables
df = df.merge(cond, on='ReadingDateTime')
df.info()

In [ ]:
cond_names = df.columns[-3:].tolist()
cond_names

In [ ]:
df[exp_names + cond_names].describe()

In [ ]:
df.to_csv('temp_data.csv', index=False)

# Temporarily save

In [ ]:
df = pd.read_csv('temp_data.csv')

In [ ]:
df.info()

In [ ]:
# covert the DateTime column to numpy.datetime variable
df['ReadingDateTime'] = pd.to_datetime(df['ReadingDateTime'], format="%d/%m/%Y %H:%M")
df.rename(columns={'ReadingDateTime':'DateTime'}, inplace=True)

In [ ]:
df.info()

In [ ]:
exp_names = df.columns[3:8].tolist()
exp_names[1] = 'log_Gsp_area'
exp_names[4] = 'log_Gsp_per_Rd_len'
exp_names

In [ ]:
cond_names = df.columns[-3:].tolist()

In [ ]:
loc_gdf = loc_gdf.set_index('siteid')

In [ ]:
loc_gdf = pd.merge(df.groupby('Site').mean()[exp_names], loc_gdf, left_index=True, right_index=True)

In [ ]:
# kernel weight matrix for the sites
weight = Kernel.from_dataframe(loc_gdf, geom_col='geometry', function='gaussian')

In [ ]:
for var in exp_names:
    moran_temp = Moran(loc_gdf[var].values, weight)
    print("Global Moran's I for " + var + ' is ', round(moran_temp.I, 5), 
          ' p-value: ', round(moran_temp.p_norm, 5))

In [ ]:
df['Value'].hist(bins=list(range(40)))

In [ ]:
df['log_Value'] = np.log(df['Value'])

In [ ]:
df['log_Value'].hist(bins=40)

In [ ]:
df['hour'] = df['DateTime'].dt.hour
df.groupby('hour').mean()['Value'].plot()

In [ ]:
df['dayofweek'] = df['DateTime'].dt.dayofweek
df.groupby('dayofweek').mean()['Value'].plot()

In [ ]:
df['dayofmonth'] = df['DateTime'].dt.day
df.groupby('dayofmonth').mean()['Value'].plot()

In [ ]:
df[['Value'] + exp_names + cond_names].hist()

In [ ]:
plt.hist(np.log(100-df['rh_mean']), bins=30)

In [ ]:
stats.normaltest(np.log(100-df['rh_mean']))

In [ ]:
def get_importance(reg, features, target, feature_names, rep=50, method='r2'):
    mean = []
    std = []
    importance = permutation_importance(reg, features, target, n_repeats=rep,
                                        random_state=25, scoring=method)
    for i in range(len(feature_names)):
        mean.append(round(importance.importances_mean[i], 5))
        std.append(round(importance.importances_std[i], 5))
    return mean, std

In [ ]:
def get_cv_score(reg, features, target, iter=100, split=10, method='r2'):
    score = []
    for i in range(iter):
        kf = KFold(n_splits=split, shuffle=True, random_state=i)
        cv = cross_val_score(reg, features, target, cv=kf, scoring=method).tolist()
        score = score + cv
    
    return (np.mean(score), np.std(score))

In [ ]:
reg = LinearRegression()
var = exp_names + cond_names

ap_X = df[var].values
ap_y = df['Value'].values

reg.fit(ap_X, ap_y)

get_importance(reg, ap_X, ap_y, var)

In [ ]:
get_cv_score(reg, ap_X, ap_y)

In [ ]:
df[df['hour']==0].reset_index()

In [ ]:
moran = []
for time in df['DateTime'].unique():
    moran_temp = Moran(df[df['DateTime']==time].Value.values, weight)
    moran.append([round(moran_temp.I, 5), round(moran_temp.p_norm, 5)])
moran_df = pd.DataFrame(df['DateTime'].unique(), columns=['DateTime'])
moran_df[['moran', 'p-value']] = moran
moran_df.head()

In [ ]:
moran_df['hour']= moran_df['DateTime'].dt.hour

In [ ]:
fig,ax = plt.subplots(4, 6, figsize=(24,16))
i = 0
for hour in range(24):
    sns.lineplot(x=moran_df['DateTime'].dt.date.unique(), 
                 y=moran_df[moran_df['hour']==hour].moran.values, 
                 ax=ax[i//6, i%6], linewidth=1)
    i+=1
plt.show()

In [ ]:
h_fi = []
h_cv = []
h_coef = []
for hour in df['hour'].unique():
    X = df[df['hour']==hour].loc[:,var].values
    y = df[df['hour']==hour].loc[:,'Value'].values
    reg.fit(X, y)
    
    fi_mean, fi_std = get_importance(reg, X, y, feature_names=var)
    h_fi.append(fi_mean + fi_std)
    
    r2 = list(get_cv_score(reg, X, y))
    mse = list(get_cv_score(reg, X, y, method='neg_mean_squared_error'))
    h_cv.append(r2 + mse)
    
    coef = reg.coef_.tolist()
    coef.append(reg.intercept_)
    h_coef.append(coef)
    
h_fi = pd.DataFrame(h_fi, columns=['fi_' + elem for elem in var] + ['fi_std_' + elem for elem in var])
h_cv = pd.DataFrame(h_cv, columns=['r2', 'r2_std', 'mse', 'mse_std'])
h_coef = pd.DataFrame(h_coef, columns=var+['intercept'])

In [ ]:
h_reg = pd.concat([h_coef, h_cv, h_fi], axis=1)
h_reg

In [ ]:
fig, ax = plt.subplots(4, 6, figsize=(24, 16))
i = 0
for hour in range(24):
    g = sns.barplot(x=['fi_' + elem for elem in var], y=h_reg.loc[hour, ['fi_' + elem for elem in var]],
                    ax=ax[i//6, i%6])
    g.set(xticklabels=[])
    i += 1
plt.legend()
plt.show()

In [ ]:
# set up a new column for month information
df['month'] = df['DateTime'].dt.month

In [ ]:
moran_df['month'] = moran_df['DateTime'].dt.month

In [ ]:
fig,ax = plt.subplots(3, 4, figsize=(16,12))
i = 0
for month in range(1,13):
    sns.lineplot(x='DateTime', y='moran', data=moran_df[moran_df['month']==month],
                 ax=ax[i//4, i%4], linewidth=1)
    i+=1
plt.show()

In [ ]:
m_fi = []
m_cv = []
m_coef = []
for month in df['month'].unique():
    X = df[df['month']==month].loc[:, var].values
    y = df[df['month']==month].loc[:, 'Value'].values
    reg.fit(X, y)
    
    fi_mean, fi_std = get_importance(reg, X, y, feature_names=var)
    m_fi.append(fi_mean + fi_std)
    
    r2 = list(get_cv_score(reg, X, y))
    mse = list(get_cv_score(reg, X, y, method='neg_mean_squared_error'))
    m_cv.append(r2 + mse)
    
    coef = reg.coef_.tolist()
    coef.append(reg.intercept_)
    m_coef.append(coef)
    
m_fi = pd.DataFrame(m_fi, columns=['fi_' + elem for elem in var] + ['fi_std_' + elem for elem in var])
m_cv = pd.DataFrame(m_cv, columns=['r2', 'r2_std', 'mse', 'mse_std'])
m_coef = pd.DataFrame(m_coef, columns=var+['intercept'])

In [ ]:
m_reg = pd.concat([m_coef, m_cv, m_fi], axis=1)
m_reg

In [ ]:
fig, ax = plt.subplots(3, 4, figsize=(16, 12))
i = 0
for month in range(1,13):
    g = sns.barplot(x=['fi_' + elem for elem in var], y=m_reg.loc[month-1, ['fi_' + elem for elem in var]],
                    ax=ax[i//4, i%4])
    g.set(xticklabels=[])
    i += 1
plt.legend()
plt.show()

In [ ]:
df.groupby('month').mean()['Value'].plot()

In [ ]:
df.info()

In [ ]:
high_period = df[df['month'].isin([1, 2, 3, 4])].drop(['geometry','hour','dayofweek','dayofmonth','month'],axis=1)
low_period = df[~df['month'].isin([1, 2, 3, 4])].drop(['geometry','hour','dayofweek','dayofmonth','month'],axis=1)

print('high period: '+str(high_period.shape)+'\nlow period: '+str(low_period.shape))

In [ ]:
hp_X = high_period[var].values
hp_y = high_period['Value'].values
reg.fit(hp_X, hp_y)

get_importance(reg, hp_X, hp_y, feature_names=var)

In [ ]:
get_cv_score(reg, hp_X, hp_y)

In [ ]:
reg.coef_.tolist() + [reg.intercept_]

In [ ]:
lp_X = low_period[var].values
lp_y = low_period['Value'].values
reg.fit(lp_X, lp_y)

get_importance(reg, lp_X, lp_y, feature_names=var)

In [ ]:
get_cv_score(reg, lp_X, lp_y)

In [ ]:
reg.coef_.tolist() + [reg.intercept_]

# Mean analysis

In [ ]:
hmean_df = df.groupby(['hour','Site']).mean()
hmean_df.info()

In [ ]:
hmean_df.drop(['bp_mean','tmp_mean','rh_mean','dayofweek','dayofmonth'],axis=1,inplace=True)

In [ ]:
hmean_df.hist()

In [ ]:
sns.heatmap(hmean_df[['Value']+exp_names].corr().round(4),annot=True,fmt='.4f',cmap='magma')
plt.show()

In [ ]:
def get_corr(df,iter_range,method='pearson',features=exp_names,target='Value'):
    result=[]
    for index in iter_range:
        result.append(df.loc[(index,)].corr(method=method).loc[features,target])
    result=np.asarray(result)
    return result

In [ ]:
def get_moran(df,iter_range,w=weight,target='Value'):
    result=[]
    for index in iter_range:
        result.append(Moran(df.loc[(index,),target].values,w).I)
    return result

In [ ]:
def get_reg_info(df,iter_range,features=exp_names,target='Value',reg=LinearRegression()):
    result=[]
    for index in iter_range:
        x=df.loc[(index,),features].values
        #x=(x-np.mean(x,axis=0))/np.std(x,axis=0)
        
        y=df.loc[(index,),target].values
        #y=(y-np.mean(y,axis=0))/np.std(y,axis=0)
        
        reg.fit(x,y)
        coef=reg.coef_.tolist()
        coef.append(reg.score(x,y))
        result.append(coef)
    result=pd.DataFrame(result,columns=features+['score'])
    return result

In [ ]:
hmean_corr=get_corr(hmean_df,range(24))
sns.lineplot(data=hmean_corr,legend=False)
plt.legend(labels=exp_names)
plt.show()

In [ ]:
hmean_corr_sp=get_corr(hmean_df,range(24),method='spearman')
sns.lineplot(data=hmean_corr_sp,legend=False)
plt.legend(labels=exp_names)
plt.show()

In [ ]:
hmean_moran=get_moran(hmean_df,range(24))
sns.lineplot(x=range(24),y=hmean_moran)

In [ ]:
hmean_reg=get_reg_info(hmean_df,range(24))
sns.lineplot(x=range(24),y=hmean_reg['score'])

In [ ]:
# plot monthly mean
df.groupby('month').mean()['Value'].plot()

In [ ]:
mmean_df=df.groupby(['month','Site']).mean()
mmean_df.info()

In [ ]:
mmean_corr=get_corr(mmean_df,range(1,13))
sns.lineplot(data=mmean_corr,legend=False)
plt.legend(labels=exp_names,loc='upper left')
plt.show()

In [ ]:
mmean_corr_sp=get_corr(mmean_df,range(1,13),method='spearman')
sns.lineplot(data=mmean_corr_sp,legend=False)
plt.legend(labels=exp_names,loc='upper left')
plt.show()

In [ ]:
mmean_moran=get_moran(mmean_df,range(1,13))
sns.lineplot(x=range(1,13),y=mmean_moran)

In [ ]:
mmean_reg=get_reg_info(mmean_df,range(1,13))
sns.lineplot(x=range(1,13),y=mmean_reg['score'])

In [ ]:
# identify high period and low period
high=df[df['month'].isin([1,2,3,4])].groupby('Site').mean()
low=df[~df['month'].isin([1,2,3,4])].groupby('Site').mean()

print('high: '+str(high.shape)+'\nlow: '+str(low.shape))

In [ ]:
sns.heatmap(high[['Value']+exp_names].corr().round(4),annot=True,fmt='.4f',cmap='magma')
plt.show()

In [ ]:
high_moran=Moran(high['Value'].values,weight)
round(high_moran.I,5)

In [ ]:
reg_high=LinearRegression()
y_high = high['Value'].values
x_high = high[exp_names].values
reg_high.fit(x_high, y_high)
reg_high.score(x_high,y_high)

In [ ]:
prd_high = reg_high.predict(x_high)

r = stats.pearsonr(y_high, prd_high)[0]
r2 = r**2
t, p_value = stats.kendalltau(y_high, prd_high)
print('r2 (obs): ', round(r2, 5))
print('tau (obs): ', round(t, 5))

In [ ]:
cv_r2_high = []
cv_tau_high = []

for i in range(100):
    kf = KFold(n_splits=4, shuffle=True, random_state=i)
    cvprd_high = cross_val_predict(reg_high, x_high, y_high, cv=kf)  #predict using current random folds
    
    #correlations
    r = stats.pearsonr(y_high,cvprd_high)[0]
    t, p_value = stats.kendalltau(y_high, prd_high)
    
    #append to list
    cv_r2_high.append(r**2)
    cv_tau_high.append(t)

print('r2 (cv): ', round(np.mean(cv_r2_high), 5))
print('tau (cv): ', round(np.mean(cv_tau_high),5))
print('r2 variance (cv): ', 1.96 * np.var(cv_r2_high))
print('tau variance (cv): ', 1.96 * np.var(cv_tau_high))

In [ ]:
sns.heatmap(low[['Value']+exp_names].corr().round(4),annot=True,fmt='.4f',cmap='magma')
plt.show()

In [ ]:
low_moran=Moran(low['Value'].values,weight)
round(low_moran.I,5)

In [ ]:
reg_low = LinearRegression()
y_low = low['Value'].values
x_low = low[exp_names].values
reg_low.fit(x_low, y_low)
prd_low = reg_low.predict(x_low)

r = low['Value'].corr(pd.Series(prd_low))
r2 = r**2
t = low['Value'].corr(pd.Series(prd_low), method='kendall')
print("r2 (cv): ", round(r2,3))   
print("tau (cv): ", round(t,3))

In [ ]:
sns.heatmap(all[['Value']+exp_names].corr().round(4),annot=True,fmt='.4f',cmap='magma')
plt.show()

In [ ]:
all=df.groupby('Site').mean()
all_moran=Moran(all['Value'].values,weight)
round(all_moran.I,5)

In [ ]:
y_all=(all.groupby('Site').mean()['Value'].values)
x_all=(all.groupby('Site').mean()[exp_names].values)

reg.fit(x_all,y_all)
reg.score(x_all,y_all)